In [3]:
import os

# Set Spark version
spark_version = "spark-3.4.0"
os.environ["SPARK_VERSION"] = spark_version

# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and extract Spark
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz -O spark.tgz
!tar -xzf spark.tgz -C /content

# Install findspark
!pip install -q findspark

# Set environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Initialize Spark
import findspark
findspark.init()


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

# Read the dataset into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the schema and preview the data
df.printSchema()
df.show(5)

# Save the DataFrame as a CSV in the Colab environment
df.write.csv("/content/home_sales_revised_saved.csv", header=True)

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|

In [7]:
# 2. Create a temporary view of the DataFrame.

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

# Verify that the temporary view was created
print("Temporary view 'home_sales' created successfully!")


Temporary view 'home_sales' created successfully!


In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# Query to calculate the average price for four-bedroom houses sold per year
query = """
    SELECT
        year(date) AS year_sold,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year_sold
"""

# Execute the query
avg_price_four_bedrooms = spark.sql(query)

# Show the result
avg_price_four_bedrooms.show()


+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY year_built
"""

# Execute the query
avg_price_3bed_3bath = spark.sql(query)

# Show the result
avg_price_3bed_3bath.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY year_built
"""

# Execute the query
avg_price_filtered = spark.sql(query)

# Show the result
avg_price_filtered.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
"""

# Execute the query
avg_price_view_rating = spark.sql(query)

# Show the result
avg_price_view_rating.show()

# Stop the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.8114089965820312 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
# Cache the temporary table home_sales
spark.sql("CACHE TABLE home_sales")

# Verify that the table is cached
print("Is 'home_sales' cached? ", spark.catalog.isCached("home_sales"))


Is 'home_sales' cached?  True


In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
"""

# Execute the query using the cached table
avg_price_view_rating_cached = spark.sql(query)

# Show the result
avg_price_view_rating_cached.show()

# Stop the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.7835724353790283 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("/content/home_sales_partitioned")

# Confirm the partitioned data was saved
print("Partitioned data saved successfully.")

Partitioned data saved successfully.


In [18]:
# 11. Read the parquet formatted data.
# Read the partitioned Parquet data
parquet_df = spark.read.parquet("/content/home_sales_partitioned")

# Show the schema and a preview of the data
parquet_df.printSchema()
parquet_df.show(5)

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|

In [19]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

# Verify that the temporary table was created
print("Temporary table 'home_sales_parquet' created successfully!")

Temporary table 'home_sales_parquet' created successfully!


In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query = """
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
"""

# Execute the query using the Parquet table
avg_price_view_rating_parquet = spark.sql(query)

# Show the result
avg_price_view_rating_parquet.show()

# Stop the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.6430542469024658 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

# Verify that the table is no longer cached
print("Is 'home_sales' cached? ", spark.catalog.isCached("home_sales"))

Is 'home_sales' cached?  False


In [22]:
# 15. Check if the home_sales is no longer cached
print("Is 'home_sales' cached? ", spark.catalog.isCached("home_sales"))


Is 'home_sales' cached?  False
